In [ ]:
import requests
from bs4 import BeautifulSoup
import inspect
import json
import os   
from pathlib import Path
import pandas as pd
import re

<h3>Neste trecho, uma url é analisada, retornando uma série de códigos de aves </h3>    

<p>Ainda não está finalizado. A ideia final é criar um algorítmo que possa iterar pela página, mapeando as espécies e jogando elas dentro da minha máquina</p>

In [ ]:
urlInfo="https://www.wikiaves.com.br/especies.php?t=t"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/121.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "pt-BR,pt;q=0.9",
    "Referer": "https://www.wikiaves.com.br/"
}

responseInfo = requests.get(urlInfo, headers=headers)
soup = BeautifulSoup(responseInfo.content, "html.parser")

soup = soup.find_all("table", class_="wa-table-sp table m-table m-table--head-separator-metal wa-table-hover")[0]
aves = soup.find("tbody").find_all("script")[1:]

In [ ]:
avesDf = []

for script in aves:
    texto = script.get_text(strip=True)
    
    match = re.search(r"lsp\((.*)\);", texto)

    valores = []

    for v in match.group(1).split(","):
        v.strip().strip("'") 
        valores.append(v)

    if valores[1] != " ''":
        familiaAtual = valores[1]

    else:
        valores[1] = familiaAtual
    
    avesDf.append({
        "codigo": int(valores[0].strip().strip("'")),
        "familia": valores[1].strip().strip("'") or None,
        "nome_cientifico": valores[2].strip().strip("'"),
        "nome_popular": valores[3].strip().strip("'"),
        "slug": valores[4].strip().strip("'"),
        "fotos": int(valores[5]),
        "sons": int(valores[6]),
    })

avesDf = pd.DataFrame(avesDf)

In [ ]:
FamiliaSelect = "Columbidae"

In [ ]:
avesSelect = avesDf[
    (avesDf["familia"] == FamiliaSelect) &
    (avesDf["sons"] > 100)
][["codigo", "nome_popular"]]


<h3>Aqui pegamos um código de ave diretamente, e então o transformamos em uma pasta com todos os seus sons disponíveis.</h3>

In [ ]:
def createDataset(codigoAve, nomeAve, dwldLimit):

    urlFiles = "https://www.wikiaves.com.br//getRegistrosJSON.php?tm=s&t=s&s="+str(codigoAve)+"&o=mp&o=mp&p=1"

    #Essa url chama o endpoint /getRegistrosJSON.php
    #Os seus parâmetros, não conheço os detalhes de todos ainda
    #no entanto, eu sei que tm=s significa tipo mídia = som

    responseFiles = requests.get(urlFiles, headers=headers)

    cont = 0
    data = json.loads(responseFiles.text)["registros"]["itens"]

    if not Path("../dataset/"+nomeAve).exists(): os.makedirs("dataset/"+nomeAve)

    for registro in data:
        link = data[registro]["link"]

        #ainda não descobri como pegar diretamente o link dos sons em mp3,
        #ao invés dos espectogramas diretamente, por isso, ao substituir
        #na url mp3 por jpg, podemos pegar os sons brutos
        link = link.replace(".jpg", ".mp3")

        #por algum motivo, os links do s3 bucket com # são protegisdos,
        #e sem o # são totalmente livres
        link = link.replace("#", "")
        responseImg = requests.get(link)
        
        with open("../dataset/"+nomeAve+"/"+nomeAve+str(cont)+".mp3", 'wb') as f:
            f.write(responseImg.content)

        if cont >= dwldLimit:
            break

        cont+=1

Vamos usar a função que criamos, e iterar por avesSelect, assim fazedndo os downloads

In [ ]:
for x in range(0, len(avesSelect)):
    createDataset(avesSelect.iloc[x]["codigo"], avesSelect.iloc[x]["nome_popular"], 10)